In [2]:
%pip install --quiet PyPDF2
%pip install --quiet pdfreader
!pip install --quiet google-cloud-discoveryengine
!pip install --upgrade --quiet google-cloud-storage
!pip3 install --upgrade --quiet google-cloud-documentai
!pip3 install --upgrade --quiet google-cloud-storage
!pip3 install --upgrade --quiet google-cloud-documentai-toolbox
%pip install --upgrade --quiet  google-cloud-documentai
%pip install --upgrade --quiet google-cloud-aiplatform -q
%pip install tqdm -q
%pip install langchain -q
!pip install --upgrade --quiet  langchain-google-genai
%pip install --upgrade --quiet  google-cloud-documentai
%pip install --upgrade --quiet  google-cloud-documentai-toolbox
%pip install --upgrade --quiet  langchain-core langchain-google-vertexai
! pip3 install --upgrade --user google-cloud-aiplatform pymupdf rich

#Restart kernel after installs so that your environment can access the new packages
import IPython
import time

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

time.sleep(10)

print("Installation done")

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-documentai-toolbox 0.13.3a0 requires google-api-core<3.0.0dev,>=2.15.0, but you have google-api-core 1.34.1 which is incompatible.
google-cloud-storage 2.16.0 requires google-api-core<3.0.0dev,>=2.15.0, but you have google-api-core 1.34.1 which is incompatible.
pandas-gbq 0.23.0 requires google-api-core<3.0.0dev,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
Note: you may need to restart the

In [1]:
#no spaces or special characters allowed), ensure that it is unique
import socket
import re

UNIQUE_PREFIX = socket.gethostname()

UNIQUE_PREFIX = re.sub('[^A-Za-z0-9]+', '', UNIQUE_PREFIX)

PREFIX_NUM_ONLY=int(str(re.search(r'\d+', UNIQUE_PREFIX).group()))

REGION_ALLOCATE=PREFIX_NUM_ONLY%3

In [2]:
PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

SVC_ACC = !(gcloud config get-value core/account)
SVC_ACC = SVC_ACC[0]

PROJECT_NUMBER=str(re.search(r'\d+', SVC_ACC).group())

LOCATION="asia-southeast1"

FOLDER_NAME="."

In [3]:
from google.cloud import storage

client = storage.Client()

GCS_BUCKET_LOCATION = "asia-southeast1"

GCS_BUCKET_NAME = f"{PROJECT_ID}-{UNIQUE_PREFIX}"
GCS_BUCKET_URI = f"gs://{GCS_BUCKET_NAME}"

bucket = storage.Bucket(client, GCS_BUCKET_NAME)


if bucket.exists()==False:
    # Create a Cloud Storage Bucket
    !gcloud storage buckets create $GCS_BUCKET_URI --location=$GCS_BUCKET_LOCATION

    # Upload the PDFs located in the DatPOC_Input/ directory into the GCS bucket that you created
    !gsutil cp -r $FOLDER_NAME/POC_Input/* $GCS_BUCKET_URI/POC_Input

    # Verify that all Books 1 to 7 are uploaded to the GCS bucket (8 files in total, 2 for Part 1)
    !gsutil ls $GCS_BUCKET_URI
else:
    # Upload the PDFs located in the POC_Input/ directory into the GCS bucket that you created
    !gsutil cp -n $FOLDER_NAME/POC_Input/* $GCS_BUCKET_URI/POC_Input
    
    print(f"\n{GCS_BUCKET_NAME} already exists. Contents:\n")
    
    # Verify that all Books 1 to 7 are uploaded to the GCS bucket (8 files in total, 2 for Part 1)
    !gsutil ls $GCS_BUCKET_URI
    
def gcs_file(blob_name):
    return bucket.blob(blob_name)

Skipping existing item: gs://ai-sandbox-company-26-instance20240524104120/POC_Input/100001355_V2.pdf
Skipping existing item: gs://ai-sandbox-company-26-instance20240524104120/POC_Input/100074636_V2.pdf
Skipping existing item: gs://ai-sandbox-company-26-instance20240524104120/POC_Input/100074650_V2.pdf
Skipping existing item: gs://ai-sandbox-company-26-instance20240524104120/POC_Input/100074660_V2.pdf

ai-sandbox-company-26-instance20240524104120 already exists. Contents:

gs://ai-sandbox-company-26-instance20240524104120/Data/
gs://ai-sandbox-company-26-instance20240524104120/POC_Input/


In [4]:
from pdfreader import PDFDocument, SimplePDFViewer
from typing import Optional
from google.api_core.client_options import ClientOptions
from google.cloud import discoveryengine

In [5]:
import json

from IPython.display import display, display_pdf, IFrame

from vertexai.generative_models import (
    GenerativeModel,
    HarmCategory,
    HarmBlockThreshold,
    Part,
)

In [6]:
model = GenerativeModel("gemini-1.5-pro-preview-0409")

generation_config = {
    "temperature": 0.0,
    "response_mime_type": "application/json",
}
safety_settings = {
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH
}

In [7]:
def get_file_bytes(file_path: str) -> bytes:
    with open(file_path, "rb") as file:
        return file.read()


def get_url_from_gcs(gcs_uri: str) -> str:
    # converts gcs uri to url for image display.
    return "https://storage.googleapis.com/" + gcs_uri.replace("gs://", "").replace(
        " ", "%20"
    )


def print_multimodal_prompt(contents: list):
    """
    Given contents that would be sent to Gemini,
    output the full multimodal prompt for ease of readability.
    """
    for content in contents:
        if isinstance(content, Part):
            if content.inline_data:
                display_pdf(content.inline_data.data)
            elif content.file_data:
                display(
                    IFrame(
                        get_url_from_gcs(content.file_data.file_uri),
                        width=600,
                        height=300,
                    )
                )
        else:
            print(content)


# Send Google Cloud Storage Document to Google Cloud Storage
def process_document(
    prompt: str,
    file_uri: str,
    mime_type: str = "application/pdf",
    print_prompt: bool = False,
    print_raw_response: bool = False,
) -> str:
    # Load file directly from Google Cloud Storage
    file_part = Part.from_uri(
        uri=file_uri,
        mime_type=mime_type,
    )

    # Load contents
    contents = [file_part, prompt]

    # Send to Gemini
    response = model.generate_content(
        contents, generation_config=generation_config, safety_settings=safety_settings
    )

    if print_prompt:
        print("-------Prompt--------")
        print_multimodal_prompt(contents)

    if print_raw_response:
        print("\n-------Raw Response--------")
        print(response)

    return response.text

In [8]:
# Function to create Document AI Processor
def create_processor(project_id, location, processor_display_name, processor_type):
    # You must set the api_endpoint if you use a location other than 'us'.
    opts = ClientOptions(api_endpoint=f"{location}-documentai.googleapis.com")

    client = documentai.DocumentProcessorServiceClient(client_options=opts)

    # The full resource name of the location
    # e.g.: projects/project_id/locations/location
    parent = client.common_location_path(project_id, location)

    # Create a processor
    processor = client.create_processor(
        parent=parent,
        processor=documentai.Processor(
            display_name=processor_display_name, type_=processor_type
        ),
    )
    
    processor_id = processor.name.split('/')[-1]

    # Print the processor information
    print(f"Processor Name: {processor.name}")
    print(f"Processor Display Name: {processor.display_name}")
    print(f"Processor ID: {processor_id}")
    print(f"Processor Type: {processor.type_}")
    
    
    return processor, processor_id

#Function to retrieve list of existing processors
def list_processors(project_id: str, location: str) -> None:
    processorList=[]
    
    # You must set the api_endpoint if you use a location other than 'us'.
    opts = ClientOptions(api_endpoint=f"{location}-documentai.googleapis.com")

    client = documentai.DocumentProcessorServiceClient(client_options=opts)

    # The full resource name of the location
    # e.g.: projects/project_id/locations/location
    parent = client.common_location_path(PROJECT_ID, location)

    # Make ListProcessors request
    processor_list = client.list_processors(parent=parent)

    # Print the processor information
    for processor in processor_list:
        # print(f"Processor Name: {processor.name}")
        # print(f"Processor Display Name: {processor.display_name}")
        # print(f"Processor Type: {processor.type_}")
        processorList.append(processor)
        
    return processorList

In [9]:
from google.api_core.client_options import ClientOptions
from google.cloud import documentai

GCP_PROJECT = PROJECT_ID #'cloud-llm-preview1'
GCP_REGION='asia-southeast1'

# Variables for Document AI OCR Processor
PROCESSOR_DISPLAY_NAME = UNIQUE_PREFIX + '-ocr-processor' # Must be unique per project, e.g.: 'My Processor'
PROCESSOR_TYPE = 'OCR_PROCESSOR' # Use fetch_processor_types to get available processor types

In [10]:
LOCATION="us"

processorList=list_processors(PROJECT_ID,LOCATION)

def createUniqueProcessor():
    if len(processorList)==0:
        PROCESSOR, PROCESSOR_ID = create_processor(PROJECT_ID, LOCATION,PROCESSOR_DISPLAY_NAME, PROCESSOR_TYPE)
        return PROCESSOR, PROCESSOR_ID
    else:
        for processor in processorList:
            if PROCESSOR_DISPLAY_NAME==processor.display_name:
                PROCESSOR_ID=processor.name.split('/')[-1]
                PROCESSOR=processor
                return PROCESSOR, PROCESSOR_ID
            else:
                try:
                    PROCESSOR, PROCESSOR_ID = create_processor(PROJECT_ID, LOCATION,PROCESSOR_DISPLAY_NAME, PROCESSOR_TYPE)
                    return PROCESSOR, PROCESSOR_ID
                except:
                    continue
                

PROCESSOR, PROCESSOR_ID = createUniqueProcessor()


print(f"Processor {PROCESSOR_ID} already exists.")

Processor 150f0d629d598ba7 already exists.


In [11]:
from google.api_core.client_options import ClientOptions
from google.cloud import documentai
import pandas as pd
import os

# PROJECT_ID = "YOUR_PROJECT_ID"
GCP_PROJECT = PROJECT_ID  # 'cloud-llm-preview1'
LOCATION = "us"  # Format is 'us' or 'eu'
PROCESSOR_ID = PROCESSOR_ID  # Create processor in Cloud Console
GCP_REGION = "asia-southeast1"

# The directory containing the files
DATA_FOLDER = "POC_Input"

# Instantiates a client
docai_client = documentai.DocumentProcessorServiceClient(
    client_options=ClientOptions(api_endpoint=f"{LOCATION}-documentai.googleapis.com")
)

# The full resource name of the processor, e.g.:
# projects/project-id/locations/location/processor/processor-id
# You must create new processors in the Cloud Console first
RESOURCE_NAME = docai_client.processor_path(GCP_PROJECT, LOCATION, PROCESSOR_ID)

# Function to process a single file and return the text
def process_file(file_path):
    with open(file_path, "rb") as file:
        image_content = file.read()

    raw_document = documentai.RawDocument(content=image_content, mime_type="application/pdf")
    request = documentai.ProcessRequest(name=RESOURCE_NAME, raw_document=raw_document)
    result = docai_client.process_document(request=request)
    document_object = result.document
    text = document_object.text.replace(',', '')  # Remove all commas
    return text

# List to store the content of each file
documents_text = []

# Iterate through files in the Data folder
for file_name in os.listdir(DATA_FOLDER):
    if file_name.endswith(".pdf"):
        file_path = os.path.join(DATA_FOLDER, file_name)
        file_text = process_file(file_path)
        documents_text.append(file_text)

# Create a DataFrame with each row representing the text of a file
df_test = pd.DataFrame(documents_text, columns=["Document_Text"])

# Print the DataFrame
print(df_test)


                                       Document_Text
0  Trailblazer Demo2 Pte Ltd\n18 Kaki Bukit Road ...
1  Trailblazer Demo1 Pte Ltd\n18 Kaki Bukit Road ...
2  Trailblazer Demo4 Pte Ltd\n18 Kaki Bukit Road ...
3  Trailblazer Demo3 Pte Ltd\n18 Kaki Bukit Road ...


In [1]:
df_test['Document_Text'][2]

NameError: name 'df_test' is not defined

In [12]:
import sys

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

In [13]:
# Define project information

import sys

PROJECT_ID = ""  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

# if not running on colab, try to get the PROJECT_ID automatically
if "google.colab" not in sys.modules:
    import subprocess

    PROJECT_ID = subprocess.check_output(
        ["gcloud", "config", "get-value", "project"], text=True
    ).strip()

print(f"Your project ID is: {PROJECT_ID}")

Your project ID is: ai-sandbox-company-26


In [14]:
import sys

# Initialize Vertex AI
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [15]:
from rich import print as rich_print
from rich.markdown import Markdown as rich_Markdown
from IPython.display import Markdown, display
from vertexai.generative_models import (
    Content,
    GenerationConfig,
    GenerationResponse,
    GenerativeModel,
    HarmCategory,
    HarmBlockThreshold,
    Image,
    Part,
)
from vertexai.language_models import TextEmbeddingModel
from vertexai.vision_models import MultiModalEmbeddingModel

In [16]:
# download documents and images used in this notebook - will take ~30 sec
!gsutil -m -q rsync -r gs://github-repo/rag/intro_multimodal_rag .
print("Download completed")


from utils.multimodal_qa_with_rag_utils import (
    get_document_metadata,
    set_global_variable,
)

from utils.multimodal_qa_with_rag_utils import (
    get_similar_text_from_query,
    print_text_to_text_citation,
    get_similar_image_from_query,
    print_text_to_image_citation,
    get_gemini_response,
    display_images,
    get_answer_from_qa_system,
)

Download completed


In [17]:
# Multimodal models: Choose based on your performance/cost needs
multimodal_model_15 = GenerativeModel(
    "gemini-1.5-pro-preview-0409"
)  # works with text, code, images, video(with or without audio) and audio(mp3) with 1M input context

multimodal_model_15_v2 = GenerativeModel(
    "gemini-1.5-pro-001"
)

multimodal_model_15_flash = GenerativeModel(
    "gemini-1.5-flash-001"
)

In [18]:
# # All relevant text chunk found across documents based on user query
# context = df_test['Document_Text'][2]
# query = """format this invoice and output it in csv format at item level, keep a null value for each column that have no info, make sure the number of column and value is the same number, duplicate the PO No and Invoice No for each item:
# PO No, Invoice No, Inv Delivery No, Year, Inv Item Description, Inv Item Qty, Inv Item Price, Inv Amount
# """
# prompt = f"""Answer the question with the given context.
# Question: {query}
# Context: {context}
# Answer:
# """

# safety_settings = {
#     HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
#     HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
#     HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
#     HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
# }

# # %%time
# import pandas as pd
# from io import StringIO

# # Assuming `prompt`, `get_gemini_response`, `multimodal_model_15`, `safety_settings`, and `GenerationConfig` are defined

# # Generate response with Gemini 1.5 Pro
# response = get_gemini_response(
#     multimodal_model_15,
#     model_input=prompt,
#     stream=True,
#     safety_settings=safety_settings,
#     generation_config=GenerationConfig(temperature=1, max_output_tokens=8192),
# )

# # Extract the CSV part of the response
# csv_data = response.split("**** Result: *****")[-1].strip()

# # Convert the CSV string to a pandas DataFrame
# data = StringIO(csv_data)
# outputdf = pd.read_csv(data)


# print(csv_data)

In [19]:
# print(outputdf)

In [20]:
import pandas as pd
from io import StringIO

# Define your query
query = """format this invoice and output it in csv format at item level, keep a null value for each column that have no info, make sure the number of column and value is the same number, duplicate the PO No and Invoice No for each item:
PO No, Invoice No, Inv Delivery No, Year, Inv Item Description, Inv Item Qty, Inv Item Price, Inv Amount
"""
# Function to get the formatted response for each document text
def get_formatted_response(document_text):
    prompt = f"""Answer the question with the given context.
    Question: {query}
    Context: {document_text}
    Answer:
    """
    
    safety_settings = {
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
    }

    # Generate response with Gemini 1.5 Pro
    response = get_gemini_response(
        multimodal_model_15,
        model_input=prompt,
        stream=True,
        safety_settings=safety_settings,
        generation_config=GenerationConfig(temperature=1, max_output_tokens=8192),
    )
    # Extract the CSV part of the response
    csv_data = response.split("**** Result: *****")[-1].strip()
    return csv_data

# Initialize an empty DataFrame to hold the combined results
combined_df = pd.DataFrame()

# Iterate over each document text in df_test
for document_text in df_test['Document_Text']:
    csv_data = get_formatted_response(document_text)
    
    # Clean the CSV data
    csv_data = csv_data.replace("```csv", "").replace("```", "").strip()
    
    # Convert the CSV string to a pandas DataFrame
    data = StringIO(csv_data)
    temp_df = pd.read_csv(data)
    
    # Append the temp_df to the combined_df
    combined_df = pd.concat([combined_df, temp_df], ignore_index=True)

# Display the combined DataFrame
combined_df


,PO No,Invoice No,Inv Delivery No,Year,Inv Item Description,Inv Item Qty,Inv Item Price,Inv Amount
0,2230109283,100074636,2.217930e+06,2024,OVERHAUL BX AFES CYL S/N 01311M,2,$4116.00,$8232.00
1,2230109283,100074636,2.217930e+06,2024,OVERHAUL BX AFES CYL S/N 01309M,2,$4116.00,$8232.00
2,1130388639,100074636,2.217920e+06,2024,SUNDSTROM BATTERY FOR SR500 (R06-0108),2,415.0,830.0
3,2830009990,100001355,9.897000e+09,2024,AIR DRIER,9,398.47,3586.23
4,2830009990,100001355,9.897000e+09,2024,SUPPLY MODULE D06,2,2118.2,4236.4
5,2230109285,100074660,NaN,2024,OVERHAUL BX AFES CYL S/N 01311M,3,$4116.00,$12348.00
6,2230109285,100074660,NaN,2024,OVERHAUL BX AFES CYL S/N 01309M,2,$4116.00,$8232.00


In [21]:
Invoice_bigtable = combined_df
print(Invoice_bigtable) 

Invoice_bigtable.to_csv('POC_Invoice_Data/Big Invoice table.csv', index=False, mode='w')

        PO No  Invoice No  Inv Delivery No  Year  \
0  2230109283   100074636     2.217930e+06  2024   
1  2230109283   100074636     2.217930e+06  2024   
2  1130388639   100074636     2.217920e+06  2024   
3  2830009990   100001355     9.897000e+09  2024   
4  2830009990   100001355     9.897000e+09  2024   
5  2230109285   100074660              NaN  2024   
6  2230109285   100074660              NaN  2024   

                     Inv Item Description  Inv Item Qty Inv Item Price  \
0         OVERHAUL BX AFES CYL S/N 01311M             2       $4116.00   
1         OVERHAUL BX AFES CYL S/N 01309M             2       $4116.00   
2  SUNDSTROM BATTERY FOR SR500 (R06-0108)             2          415.0   
3                               AIR DRIER             9         398.47   
4                       SUPPLY MODULE D06             2         2118.2   
5         OVERHAUL BX AFES CYL S/N 01311M             3       $4116.00   
6         OVERHAUL BX AFES CYL S/N 01309M             2       $41

In [22]:
import pandas as pd

# Replace 'your_csv_file.csv' with the actual path to your CSV file
PO_bigtable = pd.read_csv('POC_PO_Data/Big PO table.csv')
print(PO_bigtable)

# Replace 'your_csv_file.csv' with the actual path to your CSV file
Invoice_bigtable = pd.read_csv('POC_Invoice_Data/Big Invoice table.csv')
print(PO_bigtable)



         PO No  Item No A    Material No  \
0   1130388639         10  Not Available   
1   1130388639         10  Not Available   
2   2230109283         10  Not Available   
3   2230109283         20  Not Available   
4   2230109283         30  Not Available   
5   2230109283         40  Not Available   
6   2230109285         10  Not Available   
7   2230109285         20  Not Available   
8   2230109285         30  Not Available   
9   2230109285         40  Not Available   
10  2830009990         10  81-38100-6777   
11  2830009990         20  81-52102-6410   
12  2830009990         30  81-15403-6146   

                               Description  PO Item Qty  PO Item Price  \
0   SUNDSTROM BATTERY FOR SR500 (R06-0108)            4         415.00   
1   SUNDSTROM BATTERY FOR SR500 (R06-0108)            4         415.00   
2          OVERHAUL BX AFES CYL S/N 01311M            4        4116.00   
3          OVERHAUL BX AFES CYL S/N 01309M            6        4116.00   
4            

In [23]:

# # Filter the DataFrame for the specified invoice value
# filtered_Invoice_bigtable = Invoice_bigtable[Invoice_bigtable['PO No'] == 2830009990]
# print(filtered_Invoice_bigtable)

# # Convert the filtered DataFrame to a CSV string
# Invoice_value = filtered_Invoice_bigtable.to_csv(index=False)

# # Filter the DataFrame for the specified PO value
# filtered_PO_bigtable = PO_bigtable[PO_bigtable['PO No'] == 2830009990]
# print(filtered_PO_bigtable)

# # Convert the filtered DataFrame to a CSV string
# PO_value = filtered_PO_bigtable.to_csv(index=False)


# prompt = f"""
# PO table 
# {PO_value}

# Invoice table 
# {Invoice_value}

# Instructions:
# Ignore the PO No column but do not remove it.
# Map the invoice columns to the right if they meet the matching criteria.
# Output the comma separated values, exaclty 18 column don't give extra comma

# Matching Criteria:
# Match using 'AND' first; if not possible, then use 'OR', if still not possible then ignore:
# Description matching with Inv Item Description
# PO Item Qty matching with Inv Item Qty
# PO Item Price matching with Inv Item Price
# PO Delivery No matching Inv Delivery No
# Different material with the same unit price and quantity


# Final Table:
# Include all columns from the PO table, followed by columns from the Invoice table: Invoice No, Inv Delivery No, Year, Inv Item Description, Inv Item Qty, Inv Item Price, Inv Amount.
# """
    

# safety_settings = {
#     HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
#     HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
#     HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
#     HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
# }

# # %%time
# import pandas as pd
# from io import StringIO

# # Assuming `prompt`, `get_gemini_response`, `multimodal_model_15`, `safety_settings`, and `GenerationConfig` are defined

# # Generate response with Gemini 1.5 Pro
# response = get_gemini_response(
#     multimodal_model_15,
#     model_input=prompt,
#     stream=True,
#     safety_settings=safety_settings,
#     generation_config=GenerationConfig(temperature=1, max_output_tokens=8192),
# )

# # Extract the CSV part of the response
# csv_data = response.split("**** Result: *****")[-1].strip()
# print(csv_data)

# # Function to remove trailing commas
# def remove_trailing_commas(csv_data):
#     lines = csv_data.strip().split('\n')
#     header = lines[0]
#     processed_lines = [header]
    
#     for line in lines[1:]:
#         processed_line = line.rstrip(',')
#         processed_lines.append(processed_line)
    
#     return '\n'.join(processed_lines)

# # Clean the CSV data
# clean_csv_data = remove_trailing_commas(csv_data)

# # Convert the CSV string to a pandas DataFrame
# data = StringIO(clean_csv_data)
# outputdf_test = pd.read_csv(data)
# print(outputdf_test)
# outputdf_test.to_csv('POC_Output/outputdf_test.csv', index=False, mode='w')


In [24]:
import pandas as pd
from io import StringIO

# Function to remove trailing commas
def remove_trailing_commas(csv_data):
    lines = csv_data.strip().split('\n')
    header = lines[0]
    processed_lines = [header]
    
    for line in lines[1:]:
        processed_line = line.rstrip(',')
        processed_lines.append(processed_line)
    
    return '\n'.join(processed_lines)

# Get all unique PO Nos from PO_bigtable
unique_po_numbers = PO_bigtable['PO No'].unique()

# Initialize an empty list to store DataFrames for each PO No
dfs = []

# Step 2: Loop through the list and process each PO No
for po_no in unique_po_numbers:
    # Filter the DataFrame for the specified PO value
    filtered_PO_bigtable = PO_bigtable[PO_bigtable['PO No'] == po_no]
    print(filtered_PO_bigtable)
    print('\n')
    
    # Filter the DataFrame for the specified invoice value
    filtered_Invoice_bigtable = Invoice_bigtable[Invoice_bigtable['PO No'] == po_no]
    print(filtered_Invoice_bigtable)
    print('\n')
    
    # Convert the filtered DataFrames to CSV strings
    PO_value = filtered_PO_bigtable.to_csv(index=False)
    Invoice_value = filtered_Invoice_bigtable.to_csv(index=False)
    
    # Prepare the prompt for the Gemini model
    prompt = f"""
    PO table 
    {PO_value}

    Invoice table 
    {Invoice_value}

    Instructions:
    Ignore the PO No column but do not remove it.
    Use the PO table as the base table.
    Map the invoice columns to the right if they meet the matching criteria.
    Output the comma separated values

    Matching Criteria:
    Match using 'AND' first; if not possible, then use 'OR':
    Description matching with Inv Item Description
    PO Item Qty matching with Inv Item Qty
    PO Item Price matching with Inv Item Price
    PO Delivery No matching Inv Delivery No
    Different material with the same unit price and quantity


    Final Table:
    Include all columns from the PO table, followed by columns from the Invoice table: Invoice No, Inv Delivery No, Year, Inv Item Description, Inv Item Qty, Inv Item Price, Inv Amount.
    """
    
    safety_settings = {
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
    }
    
    # Assuming `get_gemini_response`, `multimodal_model_15`, `safety_settings`, and `GenerationConfig` are defined
    # Generate response with Gemini 1.5 Pro
    response = get_gemini_response(
        multimodal_model_15 ,
        model_input=prompt,
        stream=True,
        safety_settings=safety_settings,
        generation_config=GenerationConfig(temperature=1, max_output_tokens=8192),
    )
    
    # Extract the CSV part of the response
    csv_data = response.split("**** Result: *****")[-1].strip()
    print(csv_data)
    print('\n')

    # Clean the CSV data
    clean_csv_data = remove_trailing_commas(csv_data)

    # Convert the CSV string to a pandas DataFrame
    data = StringIO(clean_csv_data)
    df = pd.read_csv(data)
    print(df)
    print('\n')
    
    # Append the DataFrame to the list
    dfs.append(df)

# Step 3: Union all the DataFrames into one final output DataFrame
outputdf_final = pd.concat(dfs, ignore_index=True)

# Save the final output DataFrame to a CSV file
outputdf_final.to_csv('POC_Output/outputdf_final.csv', index=False, mode='w')

# Display the final DataFrame
outputdf_final

#delete the Big Invoice table

        PO No  Item No A    Material No  \
0  1130388639         10  Not Available   
1  1130388639         10  Not Available   

                              Description  PO Item Qty  PO Item Price  \
0  SUNDSTROM BATTERY FOR SR500 (R06-0108)            4          415.0   
1  SUNDSTROM BATTERY FOR SR500 (R06-0108)            4          415.0   

   PO Item Net Value Currency       GR No Item No B GR Qty PO Delivery No  
0             1660.0      SGD  5001253832         1      2        2217920  
1             1660.0      SGD  5001244644         1      2  Not Available  


        PO No  Invoice No  Inv Delivery No  Year  \
2  1130388639   100074636        2217920.0  2024   

                     Inv Item Description  Inv Item Qty Inv Item Price  \
2  SUNDSTROM BATTERY FOR SR500 (R06-0108)             2          415.0   

  Inv Amount  
2      830.0  


PO No,Item No A,Material No,Description,PO Item Qty,PO Item Price,PO Item Net Value,Currency,GR No,Item No B,GR Qty,PO Delivery No,Inv

,PO No,Item No A,Material No,Description,PO Item Qty,PO Item Price,PO Item Net Value,Currency,GR No,Item No B,GR Qty,PO Delivery No,Invoice No,Inv Delivery No,Year,Inv Item Description,Inv Item Qty,Inv Item Price,Inv Amount
0,1130388639,10,Not Available,SUNDSTROM BATTERY FOR SR500 (R06-0108),4,415.00,1660.00,SGD,5001253832,1,2,2217920,100074636.0,2.217920e+06,2024.0,SUNDSTROM BATTERY FOR SR500 (R06-0108),2.0,415.0,830.0
1,1130388639,10,Not Available,SUNDSTROM BATTERY FOR SR500 (R06-0108),4,415.00,1660.00,SGD,5001244644,1,2,Not Available,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2230109283,10,Not Available,OVERHAUL BX AFES CYL S/N 01311M,4,4116.00,16464.00,SGD,5001268120,1,2,2217930,100074636.0,2.217930e+06,2024.0,OVERHAUL BX AFES CYL S/N 01311M,2.0,$4116.00,$8232.00
3,2230109283,20,Not Available,OVERHAUL BX AFES CYL S/N 01309M,6,4116.00,24696.00,SGD,5001268120,2,2,2217930,100074636.0,2.217930e+06,2024.0,OVERHAUL BX AFES CYL S/N 01309M,2.0,$4116.00,$8232.00
4,2230109283,30,Not Available,ADD JOB: AFES BOTTLE 01311M,4,4634.00,18536.00,SGD,5001268121,1,2,2217935,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2230109283,40,Not Available,ADD JOB: AFES BOTTLE 01309M,8,4159.00,33272.00,SGD,Not Available,Not Available,Not Available,Not Available,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2230109285,10,Not Available,OVERHAUL BX AFES CYL S/N 01311M,4,4116.00,16464.00,SGD,5001268120,1,3,Not Available,100074660.0,NaN,2024.0,OVERHAUL BX AFES CYL S/N 01311M,3.0,$4116.00,$12348.00
7,2230109285,20,Not Available,OVERHAUL BX AFES CYL S/N 01309M,6,4116.00,24696.00,SGD,5001268120,2,2,Not Available,100074660.0,NaN,2024.0,OVERHAUL BX AFES CYL S/N 01309M,2.0,$4116.00,$8232.00
8,2230109285,30,Not Available,ADD JOB: AFES BOTTLE 01311M,4,4634.00,18536.00,SGD,Not Available,Not Available,Not Available,Not Available,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2230109285,40,Not Available,ADD JOB: AFES BOTTLE 01309M,8,4159.00,33272.00,SGD,Not Available,Not Available,Not Available,Not Available,NaN,NaN,NaN,NaN,NaN,NaN,NaN
